# 1. Variáveis e Pacotes que serão utilizadas

#### As seguintes variáveis serão utilizadas
- v1 = Possível score ou categoria (0, 0.693147, 1.098612, 1.386294, 1.609438, 1.791759, 1.945910, 2.079442, 2.197225)
- v2 = Possível idade (valor numérico - float)
- v3 = Possível categoria (1, 2, 3)
- v4 = Possível valor monetário (valor numérico - float)
- v6 = Valores numéricos com grande incidência do valor 0
- v7 = Valores numéricos com grande incidência do valor 0
- v8 = Valores numéricos com grande incidência do valor 0

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import RandomUnderSampler 
from collections import Counter
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [3]:
import os
os.chdir('C:/Users/DPL/Desktop/Projetos e Trabalhos/SICOOB')
os.getcwd()

'C:\\Users\\DPL\\Desktop\\Projetos e Trabalhos\\SICOOB'

In [4]:
df_completo = pd.read_excel('dados.xlsx', sheet_name='Planilha1')

In [5]:
var_selected = ['v1',
                'v2',
                'v3',
                'v4',
                'v6',
                'v7',
                'v8',
                'target']

df = df_completo[var_selected]

In [18]:
df.dtypes

v1         object
v2          int64
v3         object
v4        float64
v6         object
v7        float64
v8        float64
target      int64
v1G        object
dtype: object

# 2. Preparação dos dados
 - Transformar variáveis categóricas de float para object
 - Criação de novas variáveis
 - Tratamento de outliers

## 2.1. Transformar variáveis categóricas de float para object

In [7]:
to_object = [
            'v1',
            'v3',
            'v6']

for i in to_object:
    df[i] = df[i].astype("object")

<ipython-input-7-da6aac11c175>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = df[i].astype("object")


In [8]:
df[to_object].dtypes

v1    object
v3    object
v6    object
dtype: object

In [24]:
categorical = df.select_dtypes(include='object')

for i in categorical.columns:
    print(f"{i}: \n {df[i].value_counts().sort_index()}\n\n")

v1: 
 0.000000    7487
0.693147    1175
1.098612     875
1.386294     348
1.609438      79
1.791759      21
1.945910       8
2.079442       4
2.197225       3
Name: v1, dtype: int64


v3: 
 1    3767
2    5271
3     962
Name: v3, dtype: int64


v6: 
 0.00       9542
140.14        1
409.31        1
623.42        1
654.13        1
           ... 
2639.96       1
2674.22       1
2718.66       1
2752.96       1
3094.91       1
Name: v6, Length: 459, dtype: int64


v1G: 
 0    7487
Name: v1G, dtype: int64




## 2.2. Criação de novas vairáveis

In [19]:
df['v1G'] = df.v1.map({0.000000 : 0, 
                            0.693147 : 1,
                            1.098612 : '> 0',
                            1.386294 : '> 0',
                            1.609438 : '> 0', 
                            1.791759 : '> 0', 
                            1.945910 : '> 0',
                            2.079442 : '> 0', 
                            2.197225 : '> 0'})

In [22]:
replace_map = {'v1': {0.000000 : 0, 
                            0.693147 : 1,
                            1.098612 : '> 0',
                            1.386294 : '> 0',
                            1.609438 : '> 0', 
                            1.791759 : '> 0', 
                            1.945910 : '> 0',
                            2.079442 : '> 0', 
                            2.197225 : '> 0'    
}}

In [23]:
labels = df['v1'].astype('category').cat.categories.tolist()
replace_map_comp = {'v1' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}

print(replace_map_comp)

{'v1': {0.0: 1, 0.6931471805599453: 2, 1.0986122886681098: 3, 1.3862943611198906: 4, 1.6094379124341003: 5, 1.791759469228055: 6, 1.9459101490553132: 7, 2.0794415416798357: 8, 2.1972245773362196: 9}}


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   v1      10000 non-null  object 
 1   v2      10000 non-null  int64  
 2   v3      10000 non-null  object 
 3   v4      10000 non-null  float64
 4   v6      10000 non-null  object 
 5   v7      10000 non-null  float64
 6   v8      10000 non-null  float64
 7   target  10000 non-null  int64  
 8   v1G     7487 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 703.2+ KB
